In [1]:
import pandas as pd

books = pd.read_csv('books_cleaned.csv')
books.head()

,isbn13,isbn10,title,authors,categories,thumbnail,description,published_year,average_rating,num_pages,ratings_count,tittle_and_subtitle,tagged_description
0,9780002005883,0002005883,Gilead,Marilynne Robinson,Fiction,http://books.google.com/books/content?id=KQZCP...,A NOVEL THAT READERS and critics have been eag...,2004.0,3.85,247.0,361.0,Gilead,9780002005883 A NOVEL THAT READERS and critics...
1,9780002261982,0002261987,Spider's Web,Charles Osborne;Agatha Christie,Detective and mystery stories,http://books.google.com/books/content?id=gA5GP...,A new 'Christie for Christmas' -- a full-lengt...,2000.0,3.83,241.0,5164.0,Spider's Web: A Novel,9780002261982 A new 'Christie for Christmas' -...
2,9780006178736,0006178731,Rage of angels,Sidney Sheldon,Fiction,http://books.google.com/books/content?id=FKo2T...,"A memorable, mesmerizing heroine Jennifer -- b...",1993.0,3.93,512.0,29532.0,Rage of angels,"9780006178736 A memorable, mesmerizing heroine..."
3,9780006280897,0006280897,The Four Loves,Clive Staples Lewis,Christian life,http://books.google.com/books/content?id=XhQ5X...,Lewis' work on the nature of love divides love...,2002.0,4.15,170.0,33684.0,The Four Loves,9780006280897 Lewis' work on the nature of lov...
4,9780006280934,0006280935,The Problem of Pain,Clive Staples Lewis,Christian life,http://books.google.com/books/content?id=Kk-uV...,"""In The Problem of Pain, C.S. Lewis, one of th...",2002.0,4.09,176.0,37569.0,The Problem of Pain,"9780006280934 ""In The Problem of Pain, C.S. Le..."


In [2]:
books.categories.value_counts().reset_index()

,categories,count
0,Fiction,2111
1,Juvenile Fiction,390
2,Biography & Autobiography,311
3,History,207
4,Literary Criticism,124
...,...,...
474,Aged women,1
475,Imperialism,1
476,Human-animal relationships,1
477,Amish,1


In [3]:
category_mapping = {
    "Fiction": "Fiction",
    "Juvenile Fiction": "Children's Fiction",
    "Biography & Autobiography": "NonFiction",
    "History": "NonFiction",
    "Literature Criticism": "NonFiction",
    "Philosophy": "NonFiction",
    "Religion": "NonFiction",
    "Comics & Graphic Novels": "Fiction",
    "Drama": "Fiction",
    "Juvenile Nonfiction": "Children's NonFiction",
    "Science": "NonFiction",
    "Poetry": "Fiction"
}

books['simplified_category'] = books['categories'].map(category_mapping)

In [4]:
books.simplified_category.isna().count()

np.int64(5197)

In [8]:
from transformers import pipeline
import torch

fiction_categories = ["Fiction", "NonFiction"]
pipe = pipeline("zero-shot-classification", model="facebook/bart-large-mnli",
                device="cuda" if torch.cuda.is_available() else -1)

Device set to use cuda


In [9]:
sequence = books.loc[books['simplified_category'] == "Fiction", "description"].reset_index(drop=True)[0]

In [10]:
pipe(sequence, fiction_categories)

{'sequence': 'A NOVEL THAT READERS and critics have been eagerly anticipating for over a decade, Gilead is an astonishingly imagined story of remarkable lives. John Ames is a preacher, the son of a preacher and the grandson (both maternal and paternal) of preachers. It’s 1956 in Gilead, Iowa, towards the end of the Reverend Ames’s life, and he is absorbed in recording his family’s story, a legacy for the young son he will never see grow up. Haunted by his grandfather’s presence, John tells of the rift between his grandfather and his father: the elder, an angry visionary who fought for the abolitionist cause, and his son, an ardent pacifist. He is troubled, too, by his prodigal namesake, Jack (John Ames) Boughton, his best friend’s lost son who returns to Gilead searching for forgiveness and redemption. Told in John Ames’s joyous, rambling voice that finds beauty, humour and truth in the smallest of life’s details, Gilead is a song of celebration and acceptance of the best and the worst

In [11]:
import numpy as np

max_index = np.argmax(pipe(sequence, fiction_categories)["scores"])
max_label = pipe(sequence, fiction_categories)["labels"][max_index]
max_label

'Fiction'

In [12]:
def generate_category(sequence, categories):
    result = pipe(sequence, categories)
    max_index = np.argmax(result["scores"])
    max_label = result["labels"][max_index]
    return max_label

In [13]:
generate_category(sequence, fiction_categories)

'Fiction'

In [18]:
from tqdm import tqdm

actual_categories = []
predicted_categories = []

for i in tqdm(range(0, 300)):
    sequence = books.loc[books['simplified_category'] == 'Fiction', "description"].reset_index(drop=True)[i]
    predicted_categories += [generate_category(sequence, fiction_categories)]
    actual_categories += ['Fiction']

100%|██████████| 300/300 [00:48<00:00,  6.12it/s]


In [19]:
for i in tqdm(range(0, 300)):
    sequence = books.loc[books['simplified_category'] == 'NonFiction', "description"].reset_index(drop=True)[i]
    predicted_categories += [generate_category(sequence, fiction_categories)]
    actual_categories += ['NonFiction']

100%|██████████| 300/300 [00:47<00:00,  6.34it/s]


In [20]:
len(predicted_categories) == len(actual_categories)

True

In [21]:
predictions_df = pd.DataFrame({'actual': actual_categories, 'predicted': predicted_categories})

In [22]:
predictions_df.head()

,actual,predicted
0,Fiction,Fiction
1,Fiction,NonFiction
2,Fiction,NonFiction
3,Fiction,NonFiction
4,Fiction,NonFiction


In [25]:
predictions_df['correct_predictions'] = np.where(predictions_df['actual'] == predictions_df['predicted'], 1, 0)

In [28]:
predictions_df['correct_predictions'].sum()/len(predictions_df)

np.float64(0.685)

In [29]:
isbns = []
predicted_cats = []

missing_cats = books.loc[books['simplified_category'].isna(), ['isbn13', 'description']].reset_index(drop=True)

In [30]:
for i in tqdm(range(0, len(missing_cats))):
    sequence = missing_cats['description'][i]
    predicted_cats += [generate_category(sequence, fiction_categories)]
    isbns += [missing_cats['isbn13'][i]]

100%|██████████| 1578/1578 [05:10<00:00,  5.09it/s]


In [31]:
missing_cats_df = pd.DataFrame({'isbn13': isbns, 'predicted_categories': predicted_cats})
missing_cats_df.head()

,isbn13,predicted_categories
0,9780002261982,Fiction
1,9780006280897,NonFiction
2,9780006280934,NonFiction
3,9780006380832,NonFiction
4,9780006470229,Fiction


In [32]:
books = pd.merge(books, missing_cats_df, on='isbn13', how='left')
books['simplified_category'] = np.where(books['simplified_category'].isna(), books['predicted_categories'], books['simplified_category'])
books = books.drop(columns=['predicted_categories'])


In [33]:
books

,isbn13,isbn10,title,authors,categories,thumbnail,description,published_year,average_rating,num_pages,ratings_count,tittle_and_subtitle,tagged_description,simplified_category
0,9780002005883,0002005883,Gilead,Marilynne Robinson,Fiction,http://books.google.com/books/content?id=KQZCP...,A NOVEL THAT READERS and critics have been eag...,2004.0,3.85,247.0,361.0,Gilead,9780002005883 A NOVEL THAT READERS and critics...,Fiction
1,9780002261982,0002261987,Spider's Web,Charles Osborne;Agatha Christie,Detective and mystery stories,http://books.google.com/books/content?id=gA5GP...,A new 'Christie for Christmas' -- a full-lengt...,2000.0,3.83,241.0,5164.0,Spider's Web: A Novel,9780002261982 A new 'Christie for Christmas' -...,Fiction
2,9780006178736,0006178731,Rage of angels,Sidney Sheldon,Fiction,http://books.google.com/books/content?id=FKo2T...,"A memorable, mesmerizing heroine Jennifer -- b...",1993.0,3.93,512.0,29532.0,Rage of angels,"9780006178736 A memorable, mesmerizing heroine...",Fiction
3,9780006280897,0006280897,The Four Loves,Clive Staples Lewis,Christian life,http://books.google.com/books/content?id=XhQ5X...,Lewis' work on the nature of love divides love...,2002.0,4.15,170.0,33684.0,The Four Loves,9780006280897 Lewis' work on the nature of lov...,NonFiction
4,9780006280934,0006280935,The Problem of Pain,Clive Staples Lewis,Christian life,http://books.google.com/books/content?id=Kk-uV...,"""In The Problem of Pain, C.S. Lewis, one of th...",2002.0,4.09,176.0,37569.0,The Problem of Pain,"9780006280934 ""In The Problem of Pain, C.S. Le...",NonFiction
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5192,9788172235222,8172235224,Mistaken Identity,Nayantara Sahgal,Indic fiction (English),http://books.google.com/books/content?id=q-tKP...,On A Train Journey Home To North India After L...,2003.0,2.93,324.0,0.0,Mistaken Identity,9788172235222 On A Train Journey Home To North...,NonFiction
5193,9788173031014,8173031010,Journey to the East,Hermann Hesse,Adventure stories,http://books.google.com/books/content?id=rq6JP...,This book tells the tale of a man who goes on ...,2002.0,3.70,175.0,24.0,Journey to the East,9788173031014 This book tells the tale of a ma...,NonFiction
5194,9788179921623,817992162X,The Monk Who Sold His Ferrari: A Fable About F...,Robin Sharma,Health & Fitness,http://books.google.com/books/content?id=c_7mf...,"Wisdom to Create a Life of Passion, Purpose, a...",2003.0,3.82,198.0,1568.0,The Monk Who Sold His Ferrari: A Fable About F...,9788179921623 Wisdom to Create a Life of Passi...,Fiction
5195,9788185300535,8185300534,I Am that,Sri Nisargadatta Maharaj;Sudhakar S. Dikshit,Philosophy,http://books.google.com/books/content?id=Fv_JP...,This collection of the timeless teachings of o...,1999.0,4.51,531.0,104.0,I Am that: Talks with Sri Nisargadatta Maharaj,9788185300535 This collection of the timeless ...,NonFiction


In [34]:
books.to_csv('books_with_categories.csv', index=False)